# Requirement 1

### Structure

In [1]:
n_companies = 3
days = 10

for c in range(n_companies):
    for d in range(days):
        #choose a price p
        price = pr_agent.ChoosePrice()
        # one auction for each user starts 
        agent = UCBagent()
        bid = choose_bid() # here we implement the function to change the value of the bid in time.
        win_count = auction(n_users, n_companies, my_bid)
        agent.update()
        #for u in range (n_users):
             
        # function to determine if the user bought the product with the demand curve
        fatturato = pr_env.fatturato(n_click, demand_curve)
        
        
        
        


SyntaxError: incomplete input (2836940658.py, line 16)

### This way we change bid value many times for the same price. Then each day we chamge the price p and we try with different prices